In [1]:
import enviroments_package
import gymnasium

world_dir = "/Users/jeste/Desktop/Clase/TFG/drone_tfg_juanes/simulation_package/worlds/my_frst_webots_world.wbt"
json_take_off = "/Users/jeste/Desktop/Clase/TFG/drone_tfg_juanes/configs/reward_package_config/takeoff.json"

Archivo de contador ya existe en C:\Users\jeste\Desktop\Clase\TFG\drone_tfg_juanes\simulation_package\controllers\xyz_controller\drone_library\config\instance_counter.txt.


In [2]:
from enviroments_package import RemoveKeyObservationWrapper, ScaleRewardWrapper, ScaleActionWrapper

env = gymnasium.make('drone_tfg_juanes/Drone-v1', simulation_path = world_dir, reward_json_path = json_take_off, no_render=False)

env = RemoveKeyObservationWrapper(env, remove_keys=["camera", "gps"])
env = ScaleRewardWrapper(env, scale_factor=0.1)
env = ScaleActionWrapper(env, in_low=0, in_high=2, out_low=0, out_high=576)

Nombres generados: request_memory_7, response_memory_7


In [3]:
from stable_baselines3.common.callbacks import BaseCallback
import pandas as pd
import os
from datetime import datetime

class TrainingMetricsCallback(BaseCallback):
    def __init__(self, check_freq, log_dir, env, verbose=1):
        super(TrainingMetricsCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.metrics = []  # Guardar todas las métricas en una lista
        self.env = env

        # Crear el archivo de log con timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.log_file = os.path.join(self.log_dir, f"training_metrics_{timestamp}.csv")

    def _on_step(self) -> bool:
        return True

    def _on_rollout_end(self) -> bool:
        if self.num_timesteps % self.check_freq == 0:
            policy_loss = self.model.logger.name_to_value.get("train/policy_loss", None)
            value_loss = self.model.logger.name_to_value.get("train/value_loss", None)
            entropy = self.model.logger.name_to_value.get("train/entropy_loss", None)

            # Calcular recompensa promedio del rollout actual
            episode_reward = sum(self.locals["rewards"])
            episode_length = len(self.locals["rewards"])

            metrics_data = {
                "episode_reward": episode_reward,
                "policy_loss": policy_loss,
                "value_loss": value_loss,
                "entropy": entropy,
                "episode_length": episode_length,
            }
            self.metrics.append(metrics_data)

            # Guardar todas las métricas en un archivo CSV
            df = pd.DataFrame(self.metrics)
            df.to_csv(self.log_file, index=False)

        return True

    def _on_training_end(self):
        # Este método se llama al final del entrenamiento
        print("Entrenamiento finalizado. Cerrando el entorno...")
        self.env.close()  # Cerrar el entorno para evitar problemas


# Usar el callback al entrenar
log_dir = "./reward_logs/"
os.makedirs(log_dir, exist_ok=True)
callback = TrainingMetricsCallback(check_freq=10, log_dir=log_dir, env=env)

In [4]:
import os
from stable_baselines3 import PPO

timesteps = 2048

# Verificar si el archivo ppomodel existe en la carpeta ./models/
if not os.path.exists('./models/ppomodel'):
    # Crear y entrenar el modelo solo si el archivo no existe
    model = PPO(
        "MultiInputPolicy",
        env,
        verbose=1,
        n_steps=512,       # Tamaño típico, controla el buffer de experiencias para actualizar la política
        batch_size=32,      # Tamaño del lote, normalmente 64 o 128
        learning_rate=3e-4, # Tasa de aprendizaje típica en PPO
        ent_coef=0.05       # Coeficiente de entropía para exploración, ajuste común en PPO
    )
    model.learn(total_timesteps=timesteps, callback=callback)
    model.save('./models/ppomodel')  # Guardar el modelo para futuras ejecuciones
else:

    # Cargar el modelo guardado
    model = PPO.load("./models/ppomodel", env=env)

    # Continuar el entrenamiento desde el último punto
    model.learn(total_timesteps=timesteps, callback=callback)  # Ajusta los timesteps si quieres más entrenamiento

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5.68     |
|    ep_rew_mean     | 2.75     |
| time/              |          |
|    fps             | 0        |
|    iterations      | 1        |
|    time_elapsed    | 711      |
|    total_timesteps | 512      |
---------------------------------
WEBOTS Result: INFO: xyz_controller: Starting controller: python.exe -u xyz_controller.py
Archivo de contador ya existe en C:\Users\jeste\Desktop\Clase\TFG\drone_tfg_juanes\simulation_package\controllers\xyz_controller\drone_library\config\instance_counter.txt.
Nombres generados: request_memory_7, response_memory_7
request_memory_7 response_memory_7
Simulation Starting
{'ACTION': 'RESET', 'PARAMS': ''}
{'ACTION': 'SET_ALL_MOTORS', 'PARAMS': array([478.45026 ,  35.466534, 187.59581 , 319.03186 ], dtype=float32)}
{'ACTION': 'SET_ALL_MOTORS', 'PARAMS': array([14

Exception: crashed

In [5]:
model.save(path="./models/ppomodel")

In [ ]:
observation, _ = env.reset()

for i in range(100):  # Cambia el número de pasos según quieras observar
    action, _states = model.predict(observation)  # Usa deterministic=True para ver la política aprendida
    observation, reward, terminated, truncated, _ = env.step(action)

    if terminated:
        observation, _ = env.reset()  # Reiniciar el entorno al final del episodio

# Cerrar el entorno después de la prueba
env.close()